<a href="https://colab.research.google.com/github/sku1978/sk-share-repo/blob/main/azure_arm_resource_dependencies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies in ARM file

In [87]:
import json
import pandas as pd
from pandas import json_normalize

pd.set_option('max_rows', 99999)
pd.set_option('max_columns', 1000)
pd.set_option('max_colwidth', 400)

In [3]:
from google.colab import files
uploaded = files.upload()

Saving my_template.json to my_template.json


In [4]:
with open('my_template.json') as f:
    d = json.load(f)

In [ ]:
d['resources']

In [60]:
import re
def extract_resource_name(value):
  result=re.search("\[concat\(parameters\(\'factoryName\'\), \'(.*)\'\)\]", value)
  return result.group(1)

def extract_dependency_name(value):
    result=re.search("\[concat\(variables\(\'factoryId\'\), \'(.*)\'\)\]", str(value))
    if result:
      return result.group(1)

def extract_type_name(value):
    result=re.search("Microsoft.DataFactory\/factories(.*)", str(value))
    if result:
      return result.group(1)

In [84]:
resources = json_normalize(d['resources'])
resource_1=resources[['name', 'type', 'properties.type', 'dependsOn', 'properties.folder.name']].copy()
resource_1['name']=resource_1['name'].apply(extract_resource_name)
resource_1['type']=resource_1['type'].apply(extract_type_name)
resource_1=resource_1.explode('dependsOn')
resource_1['dependsOn']=resource_1['dependsOn'].apply(extract_dependency_name)

In [85]:
for name in set(resource_1['name']):
  df=resource_1[resource_1.dependsOn.str.contains(name, na=False)][['name', 'type']]
  df['consumedBy']=df['type']+df['name']
  resource_1.loc[resource_1.name == name, 'consumedBy']=str(list(set(df['consumedBy'])))

In [89]:
resource_1.to_csv("arm_resources.csv")
files.download("arm_resources.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
[x for x in resources.columns]